# Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))


**Demean value within condition within subject!**

# Binary choice

Columns that I want:
type
reward
valChosenMinusUnchosen_par, valChosenPlusUnchosen_par

In [ ]:
data_bc_clean_fmri = data_bc_clean %>%
  filter(fmri == 1)

Note that the number of trials for each type in the scanner is not the same across sessions and subjects.

In [ ]:
with(data_bc_clean_fmri, table(subnum, typeLeft, day))

In [ ]:
with(data_bc_clean_fmri, table(subnum, typeRight))

Add columns that you want and demean columns that will become parametric regressors

In [ ]:
data_bc_clean_fmri_preproc = data_bc_clean_fmri %>%
  group_by(subnum, day) %>%
  arrange(subnum, day, crossON) %>%
  ungroup() %>%
  select(subnum, day, leftChosen, possiblePayoffleft, possiblePayoffright, payoff, typeLeft) %>%
  mutate(type = typeLeft,
         reward = payoff,
         valueLeft = possiblePayoffleft,
         valueRight = possiblePayoffright, 
         valueChosen = ifelse(leftChosen, possiblePayoffleft, possiblePayoffright), 
         valueUnchosen = ifelse(leftChosen, possiblePayoffright, possiblePayoffleft),
         valChosenMinusUnchosen = valueChosen - valueUnchosen, 
         valChosenPlusUnchosen = valueChosen + valueUnchosen,
         choiceLeft = leftChosen,
         correct = ifelse( ( (leftChosen) & (valueLeft>valueRight) ),1, ifelse(  ( (leftChosen==0) & (valueLeft<valueRight) ),1, 0  )),
         session = ifelse(day == 3, "01", ifelse(day == 7, "02", ifelse(day == 11, "03", NA)))) %>%
  select(-leftChosen, -possiblePayoffleft, -possiblePayoffright, -payoff, -typeLeft) %>%
  group_by(subnum, day, type) %>%
  mutate(reward_dmn = reward - mean(reward),
         valueLeft_dmn = valueLeft - mean(valueLeft),
         valueRight_dmn = valueRight - mean(valueRight),
         valueChosen_dmn = valueChosen - mean(valueChosen),
         valueUnchosen_dmn = valueUnchosen - mean(valueUnchosen),
         valChosenMinusUnchosen_dmn = valChosenMinusUnchosen - mean(valChosenMinusUnchosen),
         valChosenPlusUnchosen_dmn = valChosenPlusUnchosen - mean(valChosenPlusUnchosen)) %>%
  select(-reward, -valueLeft, -valueRight, -valueChosen, -valueUnchosen, -valChosenMinusUnchosen, -valChosenPlusUnchosen) 

data_bc_clean_fmri_preproc

Check if demeaning worked as intended

In [ ]:
data_bc_clean_fmri_preproc %>%
  filter(subnum == 621 & day == 11 & type == 0) %>%
  ungroup() %>%
  select(-subnum, -day, -type, -choiceLeft, -correct, -session) %>%
  gather(key, value) %>%
  group_by(key) %>%
  summarise(mean_value = round(mean(value), 5))

Save to bids dir

`bids/sub-*/ses-*/beh`

In [ ]:
subnums = unique(data_bc_clean_fmri_preproc$subnum)
sessions = unique(data_bc_clean_fmri_preproc$session)
bids_dir = '/Users/zeynepenkavi/CpuEaters/overtrained_decisions_bidsfmri'

for (i in 1:length(subnums)){
  for (j in 1:length(sessions)){
    cur_sub = subnums[i]
    cur_ses = sessions[j]
    
    cur_bids_dir = paste0(bids_dir, '/sub-', cur_sub, '/ses-', cur_ses, '/beh')
    
    if (!file.exists(cur_bids_dir)){
      print(paste0("Creating ", cur_bids_dir))
      dir.create(file.path(cur_bids_dir))
    }
    
    cur_beh = data_bc_clean_fmri_preproc %>%
      filter(subnum == cur_sub & session == cur_ses) %>%
      ungroup() %>%
      select(-subnum, -day, -session)
    
    cur_fn = paste0(cur_bids_dir, '/sub-', cur_sub, '_ses-',cur_ses, '_task-binaryChoice_run-03_beh.tsv')
    print(paste0("Saving ", cur_fn))
    write.table(cur_beh, cur_fn, sep = '\t', row.names = F, quote = F)

    
  }
}

# Yes/No Task

In [ ]:
data_yn_clean_fmri = data_yn_clean %>%
  filter(fmri == 1)

In [ ]:
with(data_yn_clean_fmri, table(subnum, type, day))

In [ ]:
with(data_yn_clean_fmri, table(subnum, type))

In [ ]:
data_yn_clean_fmri

Add columns that you want and demean columns that will become parametric regressors

In [ ]:
data_yn_clean_fmri_preproc = data_yn_clean_fmri %>%
  group_by(subnum, day) %>%
  arrange(subnum, day, crossON) %>%
  ungroup() %>%
  group_by(subnum, day) %>%
  mutate(run = ifelse(session == min(session), "01", "02")) %>%
  select(subnum, day, type, yesChosen, possiblePayoff, payoff, reference, run, valueO, valueS, valueF) %>%
  mutate(reward = payoff,
         valueRef = reference,
         valueStim = possiblePayoff,
         valueChosen = ifelse(yesChosen, valueStim, valueRef),
         valueUnchosen = ifelse(yesChosen, valueRef, valueStim),
         valueChosenPlusUnchosen = valueChosen + valueUnchosen,
         valueChosenMinusUnchosen = valueChosen - valueUnchosen,
         choiceYes = yesChosen,
         correct = ifelse( ( (yesChosen) & (possiblePayoff > reference) ), 1, ifelse(  ( (yesChosen==0) & (possiblePayoff < reference) ),1, 0)),
         session = ifelse(day == 3, "01", ifelse(day == 7, "02", ifelse(day == 11, "03", NA)))) %>%
  select(-yesChosen, -possiblePayoff) %>%
  group_by(subnum, day, run, type) %>%
  mutate(reward_dmn = reward - mean(reward),
         valRef_dmn = valueRef - mean(valueRef),
         valStim_dmn = valueStim - mean(valueStim),
         valChosen_dmn = valueChosen - mean(valueChosen),
         valUnchosen_dmn = valueStim - mean(valueStim),
         valChosenPlusUnchosen_dmn = valueChosenPlusUnchosen - mean(valueChosenPlusUnchosen),
         valChosenMinusUnchosen_dmn = valueChosenMinusUnchosen - mean(valueChosenMinusUnchosen),
         valO_dmn = valueO - mean(valueO),
         valS_dmn = valueS - mean(valueS),
         valF_dmn = valueF - mean(valueF)) %>%
  select(-reward, -valueRef, -valueStim, -valueChosen, -valueUnchosen, -valueChosenPlusUnchosen, -valueChosenMinusUnchosen, -valueO, -valueS, -valueF) 

data_yn_clean_fmri_preproc

In [ ]:
data_yn_clean_fmri_preproc %>%
  filter(subnum == 621 & day == 11 & run == "01" & type == 0) %>%
  ungroup() %>%
  select(reward_dmn, valRef_dmn, valStim_dmn, valChosen_dmn, valUnchosen_dmn, valChosenPlusUnchosen_dmn, valChosenMinusUnchosen_dmn, valO_dmn, valS_dmn, valF_dmn) %>%
  gather(key, value) %>%
  group_by(key) %>%
  summarise(mean_value = round(mean(value), 5))

Save to bids dir

`bids/sub-*/ses-*/beh`

In [ ]:
subnums = unique(data_yn_clean_fmri_preproc$subnum)
sessions = unique(data_yn_clean_fmri_preproc$session)
runs = unique(data_yn_clean_fmri_preproc$run)
bids_dir = '/Users/zeynepenkavi/CpuEaters/overtrained_decisions_bidsfmri'

for (i in 1:length(subnums)){
  for (j in 1:length(sessions)){
    for(k in 1:length(runs)){
      cur_sub = subnums[i]
      cur_ses = sessions[j]
      cur_run = runs[k]
      
      cur_bids_dir = paste0(bids_dir, '/sub-', cur_sub, '/ses-', cur_ses, '/beh')
      
      if (!file.exists(cur_bids_dir)){
        print(paste0("Creating ", cur_bids_dir))
        dir.create(file.path(cur_bids_dir))
      }
      
      cur_beh = data_yn_clean_fmri_preproc %>%
        filter(subnum == cur_sub & session == cur_ses & run == cur_run) %>%
        ungroup() %>%
        select(-subnum, -day, -session, -run)
      
      cur_fn = paste0(cur_bids_dir, '/sub-', cur_sub, '_ses-',cur_ses, '_task-yesNo_run-', cur_run,'_beh.tsv')
      print(paste0("Saving ", cur_fn))
      write.table(cur_beh, cur_fn, sep = '\t', row.names = F, quote = F)  
    }
  }
}